<a href="https://colab.research.google.com/github/DuyguA/data_science_scripts/blob/master/fasttext_sentiment_last_round_convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!!pip3 install fasttext

['Collecting fasttext',
 '\x1b[?25l  Downloading https://files.pythonhosted.org/packages/f8/85/e2b368ab6d3528827b147fdb814f8189acc981a4bc2f99ab894650e05c40/fasttext-0.9.2.tar.gz (68kB)',
 '',
 '\x1b[K     |████▊                           | 10kB 20.7MB/s eta 0:00:01',
 '\x1b[K     |█████████▌                      | 20kB 7.0MB/s eta 0:00:01',
 '\x1b[K     |██████████████▎                 | 30kB 8.0MB/s eta 0:00:01',
 '\x1b[K     |███████████████████             | 40kB 8.7MB/s eta 0:00:01',
 '\x1b[K     |███████████████████████▉        | 51kB 7.4MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████▋   | 61kB 7.8MB/s eta 0:00:01',
 '\x1b[K     |████████████████████████████████| 71kB 4.7MB/s ',
 '\x1b[?25hRequirement already satisfied: pybind11>=2.2 in /usr/local/lib/python3.6/dist-packages (from fasttext) (2.5.0)',
 'Requirement already satisfied: setuptools>=0.7.0 in /usr/local/lib/python3.6/dist-packages (from fasttext) (47.1.1)',
 'Requirement already satisfied: numpy in /usr/lo

In [ ]:
import fasttext

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.bin.gz

--2020-06-07 08:17:55--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.tr.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4506977940 (4.2G) [application/octet-stream]
Saving to: ‘cc.tr.300.bin.gz’

cc.tr.300.bin.gz    100%[===================>]   4.20G  10.5MB/s    in 6m 56s  

2020-06-07 08:24:53 (10.3 MB/s) - ‘cc.tr.300.bin.gz’ saved [4506977940/4506977940]



In [ ]:
!gzip -d cc.tr.300.bin.gz

In [ ]:
m = fasttext.load_model("cc.tr.300.bin")

In [ ]:
dir(m)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_labels',
 '_words',
 'f',
 'get_analogies',
 'get_dimension',
 'get_input_matrix',
 'get_input_vector',
 'get_label_id',
 'get_labels',
 'get_line',
 'get_meter',
 'get_nearest_neighbors',
 'get_output_matrix',
 'get_sentence_vector',
 'get_subword_id',
 'get_subwords',
 'get_word_id',
 'get_word_vector',
 'get_words',
 'is_quantized',
 'labels',
 'predict',
 'quantize',
 'save_model',
 'set_args',
 'set_matrices',
 'test',
 'test_label',
 'words']

In [ ]:
import re

#from emoticon_smiley import replace_smileys

opponents = ["bimcell", "pttcell", "ttnet", "vodafone", "avea", "netcell", "vestelcell", "turk telekom", "turktelekom", "turknet"]
opp_regex = r"@?(" + r"|".join(opponents) + r")(\w+)?"

def replace_ceo(sentence):
  sentence = re.sub(r"(@?Kaan_?Terzio[gğ]lu|\bkaan\b)", "CEO", sentence, flags=re.I)
  return sentence

def replace_opponents(sentence):
  sentence = re.sub(opp_regex, " rakip şirket ", sentence, flags=re.I)
  return sentence

def replace_self_mentions(sentence):
  sentence = re.sub("t.?rkcll?", "turkcell", sentence)
  return sentence

def replace_entities(sentence):
  sentence = replace_ceo(sentence)
  sentence = replace_opponents(sentence)
  sentence = replace_self_mentions(sentence)
  return sentence

def replace_dots(sentence):
  sentence = sentence.replace(u"\u2026", "...").replace(u"\u2025", "..").replace(u"\u1427", ".")
  return sentence

def replace_brackets(sentence):
  sentence = sentence.replace("}", " ").replace("{", " ").replace("]", " ").replace("[", " ")
  return sentence

def replace_quotation_marks(sentence):
  rlist = [ u"\u2018", u"\u201c", u"\u201d", '"', u"\u0060", u"\u00b4", u"\u2019"]
  for r in rlist:
    sentence = sentence.replace(r, " ")
  return sentence

def replace_url(sentence):
  url_string = r"(http:\/\/www\.|https:\/\/www\.|http:\/\/|https:\/\/)?[a-z0-9]+([\-\.]{1}[a-z0-9]+)*\.[a-z]{2,5}(:[0-9]{1,5})?(\/.*)?"
  return re.sub(url_string, " websitesi ", sentence)

def replace_underscore(sentence):
  sentence = re.sub(r"[.]{3,}", "...", sentence)
  sentence = re.sub(r"(?<=\s)_(?=\s)", " ", sentence)
  sentence = re.sub(r"(?<=\w)([,:;])", r" \1", sentence)
  sentence = re.sub(r"(?<=[\w\s])([|<>/-])(?=[\w\s])" , " ", sentence)
  return sentence

def process_quotation(sentence):
  sentence = re.sub(r"(?<![.\d])\.", " .", sentence)
  sentence = re.sub(r"\.(?![.\d])", ". ", sentence)
  sentence = sentence.replace("?", " ? ").replace("!", " ! ").replace(",", ", ").replace("...", "... ")
  sentence = re.sub(r"[\s#\@&]", " ", sentence)
  return sentence

def replace_html(sentence):
  sentence = sentence.replace("&lt;", "<")
  sentence = sentence.replace("&gt;", ">")
  return sentence


def preprocess(sentence):
  sentence = sentence.strip()
  sentence = sentence.lower()
  sentence = replace_html(sentence)
  sentence = replace_url(sentence)
  sentence = replace_entities(sentence)
  sentence = replace_dots(sentence)
  sentence = replace_brackets(sentence)
  sentence = replace_quotation_marks(sentence)
  sentence = replace_underscore(sentence)
  sentence = process_quotation(sentence)
  tokens = sentence.strip().split()
  #t = replace_smileys(tokens)
  return  tokens

In [ ]:
preprocess("......")

['...']

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
sentences = []
y = []

In [ ]:
with open("all_sentiment_train.csv", "r", encoding="utf-8") as file:
    for line in file:
        l = line.strip()
        s, res = l.rsplit("\t", 1)
        sentences.append(s)
        y.append(int(res))

In [ ]:
test_sentences = []
test_y = []

with open("all_twitter_test.csv", "r", encoding="utf-8") as file:
    for line in file:
        l = line.strip()
        s, res = l.rsplit("\t", 1)
        test_sentences.append(s)
        test_y.append(int(res))
  

In [ ]:
test_cust_sentences = []
test_cust_y = []

with open("all_customer_test.csv", "r", encoding="utf-8") as file:
    for line in file:
        l = line.strip()
        s, res = l.rsplit("\t", 1)
        test_cust_sentences.append(s)
        test_cust_y.append(int(res))
  

In [ ]:
test_twit_sentences = []
test_twit_y = []

with open("all_twitter_test.csv", "r", encoding="utf-8") as file:
    for line in file:
        l = line.strip()
        s, res = l.rsplit("\t", 1)
        test_twit_sentences.append(s)
        test_twit_y.append(int(res))
  

In [ ]:
words = set()
for sentence in sequences:
    for word in sentence:
        words.add(word)

for sentence in test_cust_seqs:
  for word in sentence:
    words.add(word)

for sentence in test_twit_seqs:
  for word in sentence:
    words.add(word)


n_words = len(words)

In [ ]:
n_words

87077

In [ ]:
sequences = [preprocess(sentence) for sentence in sentences]
#test_seqs = [preprocess(t_sentence) for t_sentence in test_sentences]  

In [ ]:
test_cust_seqs = [preprocess(t_sentence) for t_sentence in test_cust_sentences]  
test_twit_seqs = [preprocess(t_sentence) for t_sentence in test_twit_sentences]  

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D, TimeDistributed
from keras.models import Model
from keras.initializers import Constant
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [ ]:
%tensorflow_version 2.x

In [ ]:
tok = Tokenizer(n_words+1, lower=True)
tok.fit_on_texts(sequences )
X_word = tok.texts_to_sequences(sequences)
X_word = pad_sequences(X_word, maxlen=MAX_LEN, padding="post")

In [ ]:
test_twitter_sequences = tok.texts_to_sequences(test_twit_seqs)
X_twitter_test = pad_sequences(test_twitter_sequences, MAX_LEN, padding="post")

X_twitter_test = np.array(X_twitter_test)
test_twit_y = np.array(test_twit_y)

In [ ]:
test_cust_sequences = tok.texts_to_sequences(test_cust_seqs)
X_cust_test = pad_sequences(test_cust_sequences, MAX_LEN, padding="post")

X_cust_test = np.array(X_cust_test)
test_cust_y = np.array(test_cust_y)

In [ ]:
max(len(sent) for sent in sequences)

339

In [ ]:
MAX_LEN =350

In [ ]:
word_index = tok.word_index
embed_size=300
embedding_matrix = np.zeros((n_words+1, embed_size))
for word, i in word_index.items():
    if i >= n_words: continue
    embedding_vector = m.get_word_vector(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [ ]:
X_word = np.array(X_word)
y = np.array(y)

In [ ]:
from keras.callbacks import TensorBoard
tensorboard_callback = TensorBoard(log_dir="wordConv", histogram_freq=1, write_graph=True, write_images=True, embeddings_freq=1)

/usr/local/lib/python3.6/dist-packages/keras/callbacks/tensorboard_v2.py:102: UserWarning: The TensorBoard callback does not support embeddings display when using TensorFlow 2.0. Embeddings-related arguments are ignored.
  warnings.warn('The TensorBoard callback does not support '


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D, TimeDistributed
from keras.models import Model
from keras.initializers import Constant
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, BatchNormalization
from keras.optimizers import Adam

weight_decay = 1e-4
num_filters=64

word_in = Input(shape=(MAX_LEN,))
x = Embedding(n_words+1, embed_size, embeddings_initializer=Constant(embedding_matrix), input_length=MAX_LEN, trainable=True)(word_in)
conv = Conv1D(num_filters, 7, activation='relu', padding='same')(x)
mp1 = MaxPooling1D(2)(conv)
conv1 = Conv1D(num_filters, 7, activation='relu', padding='same')(mp1)
mp2 = GlobalMaxPooling1D()(conv1)
dp1 = Dropout(0.5)(mp2)
dense1  = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay))(dp1)
dense2 = Dense(1, activation='sigmoid')(dense1)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model = Model(input=word_in, outputs=dense2)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`


In [ ]:
model.fit(X_word,y, batch_size=256,  validation_data= [X_twitter_test, test_twit_y], epochs=5)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 53746 samples, validate on 451 samples
Epoch 1/5
53746/53746 [==============================] - 266s 5ms/step - loss: 0.4329 - accuracy: 0.7914 - val_loss: 0.1245 - val_accuracy: 0.9601
Epoch 2/5
53746/53746 [==============================] - 261s 5ms/step - loss: 0.1439 - accuracy: 0.9528 - val_loss: 0.0404 - val_accuracy: 0.9933
Epoch 3/5
53746/53746 [==============================] - 262s 5ms/step - loss: 0.0544 - accuracy: 0.9836 - val_loss: 0.0129 - val_accuracy: 0.9978
Epoch 4/5
53746/53746 [==============================] - 264s 5ms/step - loss: 0.0285 - accuracy: 0.9922 - val_loss: 0.0076 - val_accuracy: 0.9978
Epoch 5/5
53746/53746 [==============================] - 263s 5ms/step - loss: 0.0193 - accuracy: 0.9949 - val_loss: 0.0077 - val_accuracy: 0.9978


In [ ]:
results = model.evaluate(X_cust_test, test_cust_y, batch_size=256)

2734/2734 [==============================] - 3s 948us/step


In [ ]:
results

[0.2956139805679698, 0.9261155724525452]

In [ ]:
incorrects = np.nonzero(np.round(model.predict(X_cust_test).reshape((-1,))).astype(int) != test_cust_y)

In [ ]:
len(incorrects[0]), len(test_cust_y)

(202, 2734)

In [ ]:
for i in incorrects[0]:
  print(test_cust_sentences[i], "\t", test_cust_y[i])

uzun zamandır kullanıyorum hala murekkep almadım. sankı murekkepı harcamıyor, kokluyor.. 	 1
beğenmedim.resimde daha büyüktü. 	 0
ÜRÜN GÜZEL FAKAT ÇOK KÜÇÜK VE RESİMDE DAHA GÜZEL GÖRÜNÜYOR 	 0
Ürün gayet başarılı okuma ve yazma hızı olarak, 2016 yılında f/p ürünüydü. Ah be döviz kuru sen nelere kadirsin... 	 1
yoruma ne gerek var kullanışlı değil yeterlim mi... 	 0
ürün güzel ama kurulumu zorladı.kullanma klavuzu ingilizce geldi satıcının dikkat etmesi gereken bir durum 	 1
İş görürmü görür ama cok kaliteli değil calışırken demirin sesleri çıkıyor 	 0
bu ucret bile cok derim. kokuda ozelliksiz. 	 0
Kesinlikle türkçe dublaj göndermiyorlar ona göre sipariş verin. Altyazılı. 	 0
Mouse biraz küçük, yetişkinlere için rahatsız edici bir unsur. 	 0
Biraz ağır 	 1
Kalite düşük , ip ince , 	 0
Evet, pazarlama harikası bir ürün. Japonlar yapmış.. Bravo! 	 0
Bir farkını göremedim ama hediye olarak gönderildiği için teşekkürler. 	 0
Ürün iyi fakat pes ve fifa 16 da sağ analog ile iş göremiyorsunuz

In [ ]:
examples = [
            ":)",
            ":))",
            "😚",
            "turkcell :)",
            "seni seviyorum turkcell",
            "gayet güzel",
            "güzel",
            "çok güzel",
            "çok iyi",
            "gayet güzel",
            "güzeldi",
            "güzelmiş",
            "çok güzeldi",
            "çok güzelmiş",
            "beğendim",
            "beğendik",
            "beğendi",
            "çok beğendim",
            "çok beğendik",
            "çok beğendik!!!",
            "sevdim",
            "sevdik",
            "çok sevdim",
            "hiç kötü değil",
            "kötü değil",
            "kötü değildi",
            "kötü değilmiş",
            "turkcell candır",
            "turkcell candir",
            ":(",
            ":((",
            "🙁",
            "turkcell berbat",
            "turkcell rezilsin",
            "rezilsin turkcell",
            "turkcell 🙁",
            "rezillik",
            "berbat",
            "berbattı",
            "rezildi",
            "paramızla rezil olduk",
            "kötü",
            "çok kötü",
            "çok kötü!!!!",
            "kötüydü",
            "çok kötüydü",
            "güzel değil",
            "güzel değildi",
            "iyi değil",
            "iyi değildi",
            "hiç güzel değil",
            "hiç güzel değildi",
            "pek güzel değildi",
            "pek güzel sayılmaz",
            "pek de güzel sayılmaz",
            "begenmedim",
            "beğenmedim",
            "pek beğenmedim",
            "hiç beğenmedim",
            "sevmedim",
            "sevemedim", 
            
]

In [ ]:
for ex in examples:
    tt = tok.texts_to_sequences([preprocess(ex)])
    ptt = pad_sequences(tt, MAX_LEN, padding="post")
    pred = model.predict(ptt)
    print(ex)
    print(np.round(pred[0]).astype(int))
    print("---------------")
    print("\n")

:)
[1]
---------------


:))
[1]
---------------


😚
[1]
---------------


turkcell :)
[1]
---------------


seni seviyorum turkcell
[1]
---------------


gayet güzel
[1]
---------------


güzel
[1]
---------------


çok güzel
[1]
---------------


çok iyi
[1]
---------------


gayet güzel
[1]
---------------


güzeldi
[1]
---------------


güzelmiş
[1]
---------------


çok güzeldi
[1]
---------------


çok güzelmiş
[1]
---------------


beğendim
[1]
---------------


beğendik
[1]
---------------


beğendi
[1]
---------------


çok beğendim
[1]
---------------


çok beğendik
[1]
---------------


çok beğendik!!!
[0]
---------------


sevdim
[1]
---------------


sevdik
[1]
---------------


çok sevdim
[1]
---------------


hiç kötü değil
[1]
---------------


kötü değil
[1]
---------------


kötü değildi
[1]
---------------


kötü değilmiş
[1]
---------------


turkcell candır
[1]
---------------


turkcell candir
[1]
---------------


:(
[0]
---------------


:((
[0]
---------------


In [ ]:
model.save("sentiment_model.hd5")

import json, codecs

tokenizer_json = tok.to_json()
with codecs.open('sentiment_tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=False))